# Closures : An In depth look with Python

## What is a closure ? And how to I make one ?

A closure is a function with an extended scope i.e. it can access nonglobal variables that are not defined, but accessed within its body. Closure can be usefull to keep track of a state. (Very similar to an object).

Lets try and understand the above statement by logically constructing a closure step by step with the principles we already know.

Check out this simple function definition with which we all are familiar with...

In [19]:
def foo():
    print('bar')

Lets create a function which has another function defined within in, which is returned to the caller. Python allows such programming patterns as it treats functions as first class citizens. For further details on this topic check out my other blog [First Class Functions in Python](https://sethirajat.com/first-class-functions-in-python/)

In [26]:
def foo():
    def bar():
        print('In bar')
    return bar

In [27]:
foo()  # returns the bar function

<function __main__.foo.<locals>.bar()>

We can also assign the returned `bar` function to a variable and then execute it.

In [30]:
x = foo()
print(x)  # <function foo.<locals>.bar at 0x00ED9DF8>
x()  # In bar

<function foo.<locals>.bar at 0x00C25B70>
In bar


## A simple example of Closure

Lets modify the foo method so that it defines some variable within its funcitons body.

In [41]:
def foo():
    var_in_foo = 10
    def bar():
        print(var_in_foo)
    return bar

In [42]:
x = foo()
x()

10


As you see above the value 10 gets printed, even though the function `foo` was executed before, x i.e. the `bar` function has closure over the variable `var_in_foo`. This may seem a little unusual because once the function is executed and done Python gets rid of the variables defined within it. However in this example the code keeps the value of the `var_in_foo`.

If you are interested in finding how this is working in detail you would have to understand how Cpython is implemented (may be I will do a post in future) with technique known as reference counting. In a nutshell Python will not get rid of the value/variable until someone is referring to it and here x contains reference to `var_in_foo`

## Digging a little deeper

Lets dig a little deeper using the `__closure__` and `code` object which is one of the few types used internally by the interpreter that is exposed to the user.

### `__closure__`

It is a read only attribute that contain bindings for the function’s free variables. (The term free variable refers to variables used in a function that are not local variables nor parameters of that function)

In [59]:
# Confirms that there is a closure over an integer object
x.__closure__  # returns a tuple

(<cell at 0x00EF6DD0: int object at 0x0FFC6540>,)

In [58]:
# Get the value of that variable
x.__closure__[0].cell_contents

10

### Code Object

Code objects represent byte-compiled executable Python code, or bytecode. There are certain attributes which the code object provides which are of interest to us with respect to closures.

`co_freevars` is a tuple containing the names of free variables. When we call this attribute on the x object (i.e. the `bar` function) it tells us that it has closure over the  `var_in_foo` free variable.

In [63]:
x.__code__.co_freevars

('var_in_foo',)

`co_cellvars` is a tuple containing the names of local variables that are referenced by nested functions. By calling this on the `foo` (the outer function) we see that 1 of its variable is referenced by its nested function.

In [62]:
foo.__code__.co_cellvars

('var_in_foo',)

The code object provides many such attributes, check out [standard python reference](https://docs.python.org/3.8/reference/datamodel.html) for further detials. Search for `Code objects` within the page.

## Some examples of closures

Enough with the theory for now, lets get some practical experience under our belt.

__Example 1__ : Create a greeting function for different languages.

In [82]:
def salutation(salutation = "Hello"):
    def greet(name):
        print(f'{salutation} {name}')
    return greet

In [85]:
french = salutation("Bonjour")
french("Jake")

Bonjour Jake


In [86]:
spanish = salutation("Hola")
spanish("Jake")

Hola Jake


In [87]:
english = salutation()
english("Jake")

Hello Jake


__Example 2__ : Create a function which keeps a running total using closure.

`nonlocal` is similar to the global keyword and tells python that the variable we are using is a non local variable 

In [92]:
# Running Total

def running_total(total = 0):
    def add_to(num):
        nonlocal total  
        total += num
        return total
    return add_to

In [93]:
expenses = running_total(50)

In [94]:
expenses(20)

70

In [95]:
expenses(10.2)

80.2

__Example 3 :__ Another take on the running total. You will notice that this example resembles much closer to object oriented programming.

Here we have the parent function `running_total` and 3 nested function. The `dispatcher` function is used to add the other 2 functions to it, the `dispatcher` function is then returned so that `sams_expenses` variable can call `add_to` and `print_expenses` in familiar instance (OOP) style programming.

In [140]:
def running_total(name, month, year):
    
    expenses = {}
    
    def add_to(expense_type, value):
        expenses[expense_type] = value

    def print_expenses():
        print(f'Name : {name}, Month : {month}, Year : {year}')
        total = 0
        for expense_type, value in expenses.items():
            print(f'{expense_type} : ${value}')
            total += value
        print('-' * 10)
        print(f'Total : ${total}')
        
    def dispatcher():
        pass
    
    dispatcher.add_to = add_to
    dispatcher.print_expenses = print_expenses

    return dispatcher

In [141]:
sams_expenses = running_total("Sam", "Jan", 2019)
sams_expenses.add_to("Grocery", 100)
sams_expenses.add_to("Rent", 900)
sams_expenses.print_expenses()

Name : Sam, Month : Jan, Year : 2019
Grocery : $100
Rent : $900
----------
Total : $1000


As you see the above methods look a lot like instance methods even though no class is involved. Not all languages support class construct e.g. Haskel, JavaScript (Classes in JavaScript are just syntatic sugar), however such style of programming is prevalent in such languages. There have been some random tests done where closures and marked to run faster than OOP (class/instance style) code (by as much as 8%) as their is no `self` variable involved. However you should understand that not everyone is familiar with such type of code, and such implementation may confuse people.

Moreover you loose certain object oriented functionality like inheritance etc...

## Real world usage

While not everyone may be ultra familiar with closures but they are definitely a usefull tool in your arsenal. Below are some of the areas and programming paradigms where closues are used and popular.

- `Decorators` - Closures are heavily, heavily used in decorators.
- `Single method classes` - Single method classes can be re-written using closures.
- `Functional programming` - They are popular in functional style programming.